In [2]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import time
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings("ignore")

In [4]:
Full_Data = pd.read_csv("train (1).csv")

# split data into test and train
y = Full_Data["Choice"]
X_og = Full_Data.drop("Choice", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_og, y, test_size=0.33, random_state=42)



#clf = LogisticRegression() ~ 75.1%

#parameters = {'n_neighbors':[5,10,15,20,25,30,40,50,75,100]}
#clf = GridSearchCV(KNeighborsClassifier(), parameters, cv=5)

parameters = {'n_estimators':[5,10,15,20,25,30,40,50], 'max_depth':[None, 5,10,6,7,8,9]}
clf = RandomForestClassifier() # 78.017% 25, 5
clf = GridSearchCV(clf, parameters, cv=5)

clf.fit(X_train,y_train)

conf = confusion_matrix(y_test,clf.predict(X_test))
print(conf)
        # All correct predicitions / all  ---- accuracy
print(round((conf[0][0] + conf[1][1])/len(y_test)*100,3))
#print(clf.feature_importances_)
clf.best_params_

[[694 202]
 [183 736]]
78.788


{'max_depth': 8, 'n_estimators': 50}

In [3]:
col_names = list(Full_Data.columns)
A_B = DataFrame()
for i in range(1,12):
    # Takes A - B for all columns: Pos number indicates A has more X than B
    A_B["_".join(col_names[i].split("_")[1:])] = pd.Series(Full_Data[col_names[i]] - Full_Data[col_names[i+11]])
A_B["Choice"] = Full_Data["Choice"] # 0 if B, 1 if A

In [32]:
y = A_B["Choice"]
X_ab = A_B.drop("Choice", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_ab, y, test_size=0.33, random_state=42)

#clf = LogisticRegression() ~ 74.6%
parameters = {'n_neighbors':[5,10,15,20,25,30,40,50,75,100]}
clf = GridSearchCV(KNeighborsClassifier(), parameters, cv=5)

#parameters = {'n_estimators':[5,10,15,20,25], 'max_depth':[None, 5,10,20]}
#clf = GridSearchCV(RandomForestClassifier(), parameters, cv=5) # 74.43

clf.fit(X_train,y_train)

conf = confusion_matrix(y_test,clf.predict(X_test))
print(conf)
        # All correct predicitions / all  ---- accuracy
print(round((conf[0][0] + conf[1][1])/len(y_test)*100,3))
clf.best_params_

[[672 224]
 [210 709]]
76.088


{'n_neighbors': 50}

In [15]:
# Scoring method
Scoring = Full_Data[["A_posts","B_posts","A_network_feature_1","A_network_feature_2","A_network_feature_3",
                   "B_network_feature_1","B_network_feature_2","B_network_feature_3"]]
Scoring["AFollowerRatio"] = Full_Data["A_follower_count"] / Full_Data["A_following_count"]
Scoring["AMentionRatio"] = Full_Data["A_mentions_received"] / Full_Data["A_mentions_sent"]
Scoring["BMentionRatio"] = Full_Data["B_mentions_received"] / Full_Data["B_mentions_sent"]
Scoring["BFollowerRatio"] = Full_Data["B_follower_count"] / Full_Data["B_following_count"]
Scoring["Choice"] = Full_Data["Choice"]

Scoring = Scoring.replace([np.inf, -np.inf], np.nan)
Scoring= Scoring.dropna()

y_scor = Scoring["Choice"]
X_scor = Scoring.drop("Choice", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_scor, y_scor, test_size=0.33, random_state=42)
#clf = LogisticRegression() #~ 65.2%
#clf = KNeighborsClassifier(25) #~ 71.1%
clf = RandomForestClassifier() # 74.45%

clf.fit(X_train,y_train)

conf = confusion_matrix(y_test,clf.predict(X_test))
print(conf)
        # All correct predicitions / all  ---- accuracy
print(round((conf[0][0] + conf[1][1])/len(y_test)*100,3))

In [50]:
y = Full_Data["Choice"]
X_og = Full_Data.drop("Choice", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_og, y, test_size=0.33, random_state=42)
#parameters = {'n_estimators':[5,10,15,20,25,30,40,50], 'max_depth':[None, 5,10,6,7,8,9]}
 # 78.017% 25, 5
#clf = GridSearchCV(clf, parameters, cv=5)

clf = RandomForestClassifier(n_estimators = 25, max_depth = 5)
clf.fit(X_train,y_train)

conf = confusion_matrix(y_test,clf.predict(X_test))
print(conf)
print("Accuracy: " + str(round((conf[0][0] + conf[1][1]) /len(X_test) * 100,2)) + "%\n")
print("Feature Importance")
for i,c in enumerate(X_train.columns):
    print(str(c) +" : "+ str(clf.feature_importances_[i]))

[[686 210]
 [190 729]]
Accuracy: 77.96%

Feature Importance
A_follower_count : 0.11299003843218015
A_following_count : 0.006611782838164033
A_listed_count : 0.11207479258319117
A_mentions_received : 0.07459895854607233
A_retweets_received : 0.032751623845442245
A_mentions_sent : 0.00984870381116736
A_retweets_sent : 0.003970567925548974
A_posts : 0.020647289634877507
A_network_feature_1 : 0.08380975683051839
A_network_feature_2 : 0.007770089128504299
A_network_feature_3 : 0.02326195120125917
B_follower_count : 0.05844319853987951
B_following_count : 0.005801572167754495
B_listed_count : 0.07490757922051648
B_mentions_received : 0.08009802842978025
B_retweets_received : 0.04575135794163221
B_mentions_sent : 0.021052883431614842
B_retweets_sent : 0.01640181122753738
B_posts : 0.023433836590358133
B_network_feature_1 : 0.1618511074935998
B_network_feature_2 : 0.016742959507991907
B_network_feature_3 : 0.007180110672409435


In [15]:
# calculate financal value
# Assuming no influnencers
cost = 5
profit = 10
buyingProb = .0001
buyingProb2 = .00015
cost_df = Full_Data[["Choice","A_follower_count","B_follower_count"]]
           # Revenue = #followers* prob of buying* 10                # cost
    # without analytics
profit_A_wo = cost_df["A_follower_count"] * buyingProb * profit - cost
profit_B_wo = cost_df["B_follower_count"] * buyingProb * profit - cost
    # perfect analytics                      
                                                        # two tweets              1 = A, 0 = B
profit_A_pa = (cost_df["A_follower_count"] * buyingProb2 * profit - 2*cost) * cost_df["Choice"]
profit_B_pa = (cost_df["B_follower_count"] * buyingProb2 * profit - 2*cost) * abs(1 - cost_df["Choice"])


In [16]:
# Analytics
pred = clf.predict(Full_Data.drop("Choice",axis=1))

revenue_2 = 0

for i in range(0, len(Full_Data)):
    if Full_Data.iloc[i, 0] == pred[i]:
        if Full_Data.iloc[i, 0] == 0:
            rev = buyingProb2 * Full_Data.iloc[i,12] * profit
        else:
            rev = buyingProb2 * Full_Data.iloc[i,1] * profit
    else:
        rev = 0
    revenue_2 += rev
profit_2 = revenue_2 - len(Full_Data) * 2*cost    
profit_2

7621773.756500005

In [18]:
print("Total profit without analytics is $" +str(round(sum(profit_A_wo + profit_B_wo),2)))
print("Total profit with perfect analytics is $" +str(round(sum(profit_A_pa + profit_B_pa),2)))
print("Total profit using the model is $" +str(round(profit_2,2)))

Total profit without analytics is $7289539.46
Total profit with perfect analytics is $8009852.89
Total profit using the model is $7621773.76


In [85]:
profit_A_wm

0         -0.0000
1          0.0000
2          0.9650
3         -0.0000
4         58.3835
5        418.6025
6          0.0000
7          0.0000
8          0.0000
9          0.0000
10        -0.0000
11        -0.0000
12        24.8450
13      6472.2230
14        -0.0000
15        63.0665
16        -0.0000
17         0.0000
18       387.8870
19      1075.7000
20         0.0000
21      7411.1150
22         0.0000
23        -0.0000
24         0.0000
25         0.0000
26      4152.1850
27       138.1820
28        76.7600
29        -0.0000
          ...    
5470      -0.0000
5471       0.0000
5472       0.0000
5473       0.0000
5474       0.0000
5475      86.1890
5476      -0.0000
5477     186.2750
5478      60.5090
5479      -6.1390
5480      -0.0000
5481     413.4575
5482      -0.0000
5483    4612.2575
5484    2212.0685
5485     671.7710
5486       0.0000
5487    4943.7200
5488     295.6820
5489      -0.0000
5490       0.0000
5491      -0.0000
5492      -0.0000
5493       0.0000
5494      